# 1. Introduction
 There are over 200 named neighborhoods in Atlanta, each with a unique history and culture. Sadly, for a long time these neighborhoods were isolated from each other. The Atlanta Beltline project aims to correct that by providing easy walking paths around the city. These pathways have seen exploding business and cultural growth since construction.
I am from Atlanta, GA and have always loved beer, so I am always on the look out for new beer serving establishments. But what would it look like to start my own? Would the Beltline help? Would I get drowned out by other bars or restaurants?

# 2. Business Problem
If I want to open a beer taproom in Atlanta, where should I put it? Where would I get the best value for my money while also reaching the most popular neighborhoods. By using population and FourSquare data, can I see which neighborhoods have the fewest beer serving establishments per capita?

# 3. Data Description

## 3.1 Atlanta Neighborhood Data
The City of Atlanta publishes geographic data for each of the neighborhoods within the city limits. This shapefile includes geometries. We will use only two columns from this data set:
* Name
* geometry

## 3.2 Atlanta Neighborhood Population
Wikipedia has an article with population values for Atlanta neighborhoods with popultions over 500 people. Since I probably do not want to open a bierhaus in a sparsely populated neighborhood, I accept this limitation. The page is here: [https://en.wikipedia.org/wiki/Table_of_Atlanta_neighborhoods_by_population](https://en.wikipedia.org/wiki/Table_of_Atlanta_neighborhoods_by_population).

## 3.3 FourSquare API Data
Using the FourSquare API, I will gather the number of beer serving venues in each Neighborhood. Since I am using the a free license with FourSquare I will limit the queries to venues and not menus. Expected data we will retrieve is:
* Neighbourhood : Name of the Neighbourhood
* Neighbourhood Latitude : Latitude of the Neighbourhood
* Neighbourhood Longitude : Longitude of the Neighbourhood
* Venue : Name of the Venue
* Venue Latitude : Latitude of Venue
* Venue Longitude : Longitude of Venue
* Venue Category : Category of Venue

In [73]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from config import fs_csec

In [74]:
atl_neighborhoods = gpd.read_file('Neighborhoods-shp/Neighborhoods.shp')

In [75]:
atl_neighborhoods.head()

,OBJECTID,LOCALID,NAME,GEOTYPE,FULLFIPS,LEGALAREA,EFFECTDATE,ENDDATE,SRCREF,ACRES,SQMILES,OLDNAME,NPU,CREATED_US,CREATED_DA,LAST_EDITE,LAST_EDI_1,GLOBALID,geometry
0,148,None,Atlanta University Center,Neighborhood,None,None,None,None,None,331.57,0.52,Atlanta University,T,None,None,GIS,2018-05-15,{D6CAF694-B83F-4CB9-B351-CA3E3145D0C7},"POLYGON ((2224815.250 1365864.750, 2224809.250..."
1,149,None,Hunter Hills,Neighborhood,None,None,None,None,None,322.58,0.50,Hunter Hills,K,None,None,GIS,2018-05-15,{8A4F3C84-CCC3-478C-82BA-7E4174935428},"POLYGON ((2215653.250 1369128.625, 2218199.913..."
2,150,None,Historic Westin Heights/Bankhead,Neighborhood,None,None,None,None,None,416.09,0.65,Bankhead,K,None,None,GIS,2020-03-09,{70303C29-B16F-4E33-9FEC-A33B2244D3AB},"POLYGON ((2220410.264 1373502.844, 2220408.143..."
3,151,None,English Avenue,Neighborhood,None,None,None,None,None,519.00,0.81,English Avenue,L,None,None,GIS,2018-05-15,{9B6F9DF9-B38E-46FB-993B-4D1DA42AC049},"POLYGON ((2221296.101 1375554.111, 2221601.514..."
4,152,None,Adair Park,Neighborhood,None,None,None,None,None,289.28,0.45,Adair Park,V,None,None,GIS,2018-05-15,{D8FD5CB2-4D90-4EEB-A33A-B52E6D66E345},"POLYGON ((2223163.620 1360278.221, 2223157.442..."


In [76]:
m = folium.Map([33.7490,-84.3880], zoom_start = 12)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)
folium.GeoJson(atl_neighborhoods).add_to(m)
m

In [77]:
url= "https://en.wikipedia.org/wiki/Table_of_Atlanta_neighborhoods_by_population"
wiki = requests.get(url)
wiki = pd.read_html(wiki.text)
wiki = wiki[0]
wiki

,Neighborhood,Population (2010),NPU
0,Adair Park,1331,V
1,Adams Park,1763,R
2,Adamsville,2403,H
3,Almond Park,1020,G
4,Ansley Park,2277,E
...,...,...,...
156,Westwood Terrace,733,I
157,Whittier Mill Village,617,D
158,Wildwood,1840,C
159,Wilson Mill Meadows,1096,H


In [78]:
atl_neighborhoods['POP'] = atl_neighborhoods.NAME.map(dict(zip(wiki.Neighborhood,wiki['Population (2010)'])))

In [79]:
atl_neighborhoods.shape

(244, 20)

In [80]:
atl_map = folium.Map([33.7490,-84.3880], zoom_start = 12)
scale = (atl_neighborhoods['POP'].quantile((0,0.1,0.75,0.9,0.98,1))).tolist()
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(atl_map)
folium.Choropleth(
 geo_data=atl_neighborhoods,
 name='Choropleth',
 data=atl_neighborhoods,
 columns=['NAME','POP'],
 key_on="feature.properties.NAME",
 fill_color='YlGnBu',
 threshold_scale=scale,
 fill_opacity=0.8,
 line_opacity=0.2,
 legend_name='Total Square Miles',
 smooth_factor=0
).add_to(atl_map)

In [81]:
atl_map

In [82]:
CLIENT_ID = 'CXDDIXSSG4QDUN25PUJXW4ZGNEAX24P15S40NYRZ5SE3FA2M' # your Foursquare ID
CLIENT_SECRET = fs_csec # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value